In [14]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from openpyxl.worksheet.dimensions import DimensionHolder, ColumnDimension
from keras.models import load_model

modelito = load_model('my_single_model.h5')

def ajustar_valores(valor):
    try:
        valor = float(valor)
        return max(0, min(20, valor))
    except:
        return 0

def procesar_hoja(df, nombre_hoja):
    if df.empty:
        return f"Error: La hoja '{nombre_hoja}' está completamente vacía."
    df.iloc[:, 1:] = df.iloc[:, 1:].applymap(ajustar_valores)
    return df

def guardar_en_excel(df_predicciones):
    file_path = "resultados_predicciones_unico_modelo.xlsx"
    with pd.ExcelWriter(file_path, engine='openpyxl') as writer:
        df_predicciones.to_excel(writer, sheet_name="ACADEMAI", index=False)
        for column in writer.sheets["ACADEMAI"].columns:
            max_length = 0
            column = [cell for cell in column]
            for cell in column:
                try:
                    if len(str(cell.value)) > max_length:
                        max_length = len(cell.value)
                except:
                    pass
            adjusted_width = (max_length + 2)
            writer.sheets["ACADEMAI"].column_dimensions[column[0].column_letter].width = adjusted_width

try:
    nombre_archivo = "Modelo_Excel.xlsx"
    hojas = pd.read_excel(nombre_archivo, sheet_name=None, engine='openpyxl')
    dfs = []

    for nombre_hoja, df in hojas.items():
        df = procesar_hoja(df, nombre_hoja)
        dfs.append(df)

    df_predicciones = dfs[-1][['Alumno']].copy()
    X = []
    y = []

    for idx, df in enumerate(dfs[:-1]):
        features = df.drop(columns=["Alumno"]).values.tolist()
        for curso in dfs[0].columns[1:]:
            targets = dfs[idx+1][curso].values
            X.extend(features)
            y.extend(targets)

    X = np.array(X, dtype=np.float32)
    y = np.array(y, dtype=np.float32)

    model = keras.Sequential([
        keras.layers.Dense(128, activation='relu', input_shape=(X.shape[1],)),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(32, activation='relu'),
        keras.layers.Dense(1)
    ])

    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    model.fit(X, y, epochs=100, validation_split=0.2, callbacks=[early_stopping])

    for curso in dfs[0].columns[1:]:
        predicciones = modelito.predict(dfs[-1].drop(columns=["Alumno"]).values.astype(np.float32))
        predicciones = np.clip(predicciones, 0, 20)
        df_predicciones[curso + ' Predicted'] = predicciones.flatten()

    for curso in df_predicciones.columns[1:]:
        df_predicciones[curso] = df_predicciones[curso].apply(lambda x: round(x, 2))

    guardar_en_excel(df_predicciones)
    #model.save('my_single_model.h5')

except Exception as e:
    print(f"Error al leer el archivo: {e}")


Epoch 1/100
1/1 [==============================] - 1s 671ms/step - loss: 264.8434 - mae: 13.7992 - val_loss: 206.6971 - val_mae: 14.3712
Epoch 2/100
1/1 [==============================] - 0s 22ms/step - loss: 210.2643 - mae: 12.0732 - val_loss: 183.7635 - val_mae: 13.5434
Epoch 3/100
1/1 [==============================] - 0s 24ms/step - loss: 212.6818 - mae: 12.3393 - val_loss: 164.2069 - val_mae: 12.7888
Epoch 4/100
1/1 [==============================] - 0s 24ms/step - loss: 222.9544 - mae: 12.1920 - val_loss: 144.8345 - val_mae: 11.9909
Epoch 5/100
1/1 [==============================] - 0s 24ms/step - loss: 181.0749 - mae: 11.0743 - val_loss: 128.5304 - val_mae: 11.2673
Epoch 6/100
1/1 [==============================] - 0s 22ms/step - loss: 160.9219 - mae: 10.5085 - val_loss: 117.5420 - val_mae: 10.7557
Epoch 7/100
1/1 [==============================] - 0s 24ms/step - loss: 157.1738 - mae: 10.7923 - val_loss: 107.3874 - val_mae: 10.2587
Epoch 8/100
1/1 [==============================